In [ ]:
library(dplyr)
library(data.table)
library(tidyr)
library(taxizedb)
library(ggplot2)

In [ ]:
hggns = paste(snakemake@input$od, "hgts",paste(snakemake@wildcards$stem,".txt",sep=""),sep = "/")
hggns_df = fread(hggns) %>%
    select(protein=V1, score=V2,taxid = V3)

In [ ]:
get_lineages <- function(x){
    # Print the fetched data
    #tax_info
    tt <- lapply(x, function(x){
        tax_info <- classification(x, db="ncbi")  
        df <- tax_info[[1]]
        if (is.data.frame(df)) {
                df$query <- x
                return(df)
            } else {
                return(NA)
            }

    })
    tt <- tt[!is.na(tt)]
    standard_ranks <- c("kingdom", "phylum", "class", "order", "family", "genus", "species")
    dfo <- bind_rows(tt) %>%
        filter(rank %in% standard_ranks)
    #     select(-id) %>%
    #     pivot_wider(names_from = rank, values_from = name)
    return(dfo)
}


In [ ]:
lgs <- get_lineages(hggns_df$taxid) %>%
    filter(rank == "order")

In [ ]:
lgs <- lgs %>%
    select(-rank, -id) %>%
    distinct()
lgs

In [ ]:
hggns_df2 <- hggns_df %>%
    left_join(lgs, by = c("taxid"="query")) %>%
    filter(!is.na(name))
hggns_df2

In [ ]:
inf <- fread(snakemake@input$close) %>%
    select(order,bin) %>%
    rowwise() %>%
    mutate(order = strsplit(split=" ",x = order)[[1]][[1]])


reciever_bin = sub(pattern = "nod_", replacement = "", x = snakemake@wildcards$stem)
reciever_order = filter(inf, bin==reciever_bin)$order
hggns_df3 <- hggns_df2 %>%
    left_join(inf, by = c("name"="order")) %>%
    filter(!is.na(bin)) %>%
    select(-bin) %>%
    distinct() %>%
    rename(donor = name) %>%
    mutate(
        reciever=reciever_order,
        bin = reciever_bin)
fwrite(hggns_df3, file = snakemake@output[[1]])


